### Repeat Task1 and Task2 select models and generate images for Figure 1

Note: this uses a modified version of weightwatcher that

- prints the images
- removes title from the images
- adds the log-log histogram of the ESD
- removes the ccdf (red lines)

#### Note: This NB

- needs the data for the specific models listed

- used a modified version of WW of 0.4.0.

- can in principle be reproduced using ww0.4.1, however, this has not been done yet since Figure 1 is simply illustrative

In [ ]:
import json
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import weightwatcher as ww
ww.__version__

In [ ]:
cd starting_kit;

In [ ]:
row = 2
if row == 3:
    this_task = "task2_v1"
    repeat_model = 'model_1006'
    repeat_layer = 10
elif row == 4:
    this_task = "task1_v4"
    repeat_model = 'model_543'
    repeat_layer = 7
elif row == 2:
    this_task = "task1_v4"
    repeat_model = 'model_152'
    repeat_layer = 1

plt.rcParams.update({'font.size': 16})



In [ ]:
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


In [ ]:
# %load ingestion_program/ingestion.py
import weightwatcher as ww

#=
debug_mode = 0

# Time budget
#############
# Maximum time of training in seconds PER MODEL. 
max_time_per_model = 60 * 5

# I/O defaults
##############
# If true, the previous output directory is not overwritten, it changes name
save_previous_results = False
# Use default location for the input and output data:
# If no arguments to run.py are provided, this is where the data will be found
# and the results written to. Change the root_dir to your local directory.
root_dir = "../"
default_input_dir = "./{}_data".format(this_task)
default_output_dir = "./{}_out".format(this_task)
default_program_dir = "ingestion_program"
default_submission_dir = "./ww_submit_nb"


import os
from sys import argv, path
import datetime
import glob
import inspect

the_date = datetime.datetime.now().strftime("%y-%m-%d-%H-%M")

import tensorflow as tf
# enable tf 2.0 behavior
tf.compat.v1.enable_v2_behavior()

filter_filenames = [".ds_store", ".DS_Store", "__MACOSX"]
verbose = True
version = 1

# =========================== BEGIN PROGRAM ================================

#### Check whether everything went well (no time exceeded)
execution_success = True

input_dir = default_input_dir
output_dir = default_output_dir
program_dir= default_program_dir
submission_dir= default_submission_dir


# Our libraries
path.append (program_dir)
path.append (submission_dir)

import data_io                       # general purpose input/output functions
from data_io import vprint           # print only in verbose mode
from data_manager import DataManager # load/save data and get info about them
#from complexity import complexity # complexity measure

#should_pass_submission_dir = 'program_dir' in inspect.getfullargspec(complexity).args

if debug_mode >= 4:
  print('File structure')
  data_io.list_files('..')

if debug_mode >= 4: # Show library version and directory structure
    data_io.show_dir(".")

# Move old results and create a new output directory (useful if you run locally)
if save_previous_results:
    data_io.mvdir(output_dir, output_dir+'_'+the_date) 
data_io.mkdir(output_dir) 

#### INVENTORY DATA (and sort dataset names alphabetically)
datanames = os.listdir(input_dir)
# change input dir to compensate for the single file unzipping
if 'input_data' in datanames:
    input_dir = os.path.join(input_dir, 'input_data')
    datanames = os.listdir(input_dir)
# Overwrite the "natural" order

#### DEBUG MODE: Show dataset list and STOP
if debug_mode>=3:
    data_io.show_version()
    data_io.show_io(input_dir, output_dir)
    print('\n****** Ingestion program version ' + str(version) + ' ******\n\n' + '========== DATASETS ==========\n')        	
    data_io.write_list(datanames)      
    datanames = [] # Do not proceed with learning and testing


for basename in datanames: # Loop over datasets
    if basename in filter_filenames:
        continue
    vprint( verbose,  "\n========== Ingestion program version " + str(version) + " ==========\n") 
    vprint( verbose,  "************************************************")
    vprint( verbose,  "******** Processing dataset " + basename.capitalize() + " ********")
    vprint( verbose,  "************************************************")


    # ======== Creating a data object with data, informations about it (write a new data manager for loading the models)
    vprint(verbose,  "========= Reading and converting data ==========")
    D = DataManager(basename, input_dir)

    training_data = D.load_training_data()
    complexity_value = {}
    
    models = {}
    for mid in D.model_ids:
        if mid==repeat_model:
            #if time_exceeded:
            #    complexity_value[mid] = 'EXCEEDED'
            #    continue
            tf.keras.backend.clear_session()
            model = D.load_model(mid)
            models[mid]=model
            print("--------\nMODEL ID {}\n--------\n".format(mid))

    

            watcher = ww.WeightWatcher()
            results = watcher.analyze(model=model, plot=True, min_size=20, layers=[repeat_layer])
    





### 

In [ ]:
IMG_DIR = "./paper/img"

In [ ]:
file =  "{}/fig1-{}a.png".format(IMG_DIR, row)
!mv  ww.esd1.png $file

file =  "{}/fig1-{}b.png".format(IMG_DIR, row)
!mv  ww.esd2.png $file

file =  "{}/fig1-{}c.png".format(IMG_DIR, row)
!mv  ww.esd3.png $file

file =  "{}/fig1-{}d.png".format(IMG_DIR, row)
!mv  ww.esd4.png $file